In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [3]:
df = pd.read_csv('../../data/clean_kc_house_data.csv')
df = df.drop(columns=['Unnamed: 0','date'])

In [4]:
x = df.drop(columns='price')
y = df['price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 23)

In [5]:
x_train

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15
18108,8161000220,3,2.50,1860,21876,2.0,Average,8 Good,1860,0.0,1992,98014,47.6455,-121.901,2450,21876
20609,9542000075,3,1.75,2000,14733,1.0,Good,8 Good,2000,0.0,1958,98005,47.6001,-122.178,2620,14733
13874,6189200345,4,2.75,2260,12005,1.0,Good,8 Good,2260,0.0,1956,98005,47.6342,-122.171,1870,10800
3279,1433100010,4,1.00,1730,8706,1.0,Good,7 Average,1010,720.0,1962,98058,47.4586,-122.175,1369,8418
10398,3886901795,6,5.00,2850,6600,2.0,Average,7 Average,2850,0.0,1994,98033,47.6813,-122.187,1870,9900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6175,2473530100,4,2.50,2440,7155,2.0,Average,8 Good,2440,0.0,1993,98058,47.4501,-122.126,2450,8109
9704,3630220220,4,3.50,3060,4573,2.0,Average,9 Better,2410,650.0,2012,98029,47.5522,-122.001,3170,3634
11190,4123840310,3,2.50,1810,5192,2.0,Average,8 Good,1810,0.0,1993,98038,47.3724,-122.042,1810,6200
9256,3524039224,4,2.50,3520,6773,2.5,Average,9 Better,2650,870.0,2006,98136,47.5317,-122.391,2930,6458


In [6]:
# print(x_train[[*colList]].shape, y_train[[*colList]].shape,)

NameError: name 'colList' is not defined

In [ ]:
def evaluateModel(X_tr,X_te,y_tr,y_te):
    '''
    Function takes in data, makes a model, and evaluates it.
    '''
    # Create and fit model
    model = sm.OLS(y_tr,sm.add_constant(X_tr)).fit()
    
    # Print sumary (if OLS)
    display(model.summary());
    
    # Grab predictions
    tr_preds = model.predict(sm.add_constant(X_tr))
    te_preds = model.predict(sm.add_constant(X_te))
    
    # Evaluate on train and test
    print("\nEvaluation Statistics:")
    print(f"Train R2 Score: {r2_score(y_tr, tr_preds):.4f}")
    print(f"Train MAE Score: ${mean_absolute_error(y_tr, tr_preds):.4f}")
    print(f"Train RMSE Score: ${mean_squared_error(y_tr, tr_preds, squared=False):.4f}")
    print("*"*20)
    print(f"Test R2 Score: {r2_score(y_te, te_preds):.4f}")
    print(f"Test MAE Score: ${mean_absolute_error(y_te, te_preds):.4f}")
    print(f"Test RMSE Score: ${mean_squared_error(y_te, te_preds, squared=False):.4f}")
    
    # Visualize residuals
    plt.figure(figsize=(9,5))
    plt.scatter(tr_preds, y_tr-tr_preds, label='Train')
    plt.scatter(te_preds, y_te-te_preds, label='Test')
    plt.axhline(y=0, color = 'red', label = '0')
    plt.ticklabel_format(style='plain')
    plt.xlabel('predictions')
    plt.ylabel('residuals')
    plt.legend()
    plt.show();
    
    return

In [ ]:
def graphPreds(X_tr,X_te,y_tr,y_te):
    # Visualize predictions
    plt.figure(figsize=(9,5))
    plt.scatter(X_tr,y_tr,label='Train')
    plt.scatter(X_te,y_te,label='Test')
    plt.plot(X_tr, tr_preds, label="Train Line of Best Fit", color='black')
    plt.plot(X_te,te_preds, label="Test Line of Best Fit", color = 'green')
    plt.hlines(y=y_tr.mean(),xmin=X_tr.min(), xmax=X_tr.max(), color='red',label=f'Average: {y_tr.mean():.2f}')
    plt.ylabel('target')
    plt.legend()
    plt.ticklabel_format(style='plain')
    plt.show();

In [ ]:
df['grade'].value_counts()

In [ ]:
colList = ['bedrooms','sqft_living','yr_built','condition','grade','bathroom']
catCols = ['condition','grade']

In [ ]:
encoder = OneHotEncoder(handle_unknown='error',drop='first',categories='auto')
ct = ColumnTransformer(transformers=[('ohe', encoder, catCols)],remainder='passthrough',sparse_threshold=0)
ct.fit(x_train[[*colList]])
encoded_xtrain = ct.transform(x_train[[*colList]])
encoded_xtest = ct.transform(x_test[[*colList]])

encoded_xtrain = pd.DataFrame(encoded_xtrain,columns=ct.get_feature_names())
encoded_xtest = pd.DataFrame(encoded_xtest,columns=ct.get_feature_names())

In [ ]:
encoded_xtrain

In [ ]:
colList = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','sqft_above','sqft_basement','sqft_living15','sqft_lot15']

In [ ]:
evaluateModel(x_train[['sqft_living']],x_test[['sqft_living']],y_train,y_test)

In [ ]:
print(x_train[[*colList]].shape, y_train[[*colList]].shape,)

In [ ]:
evaluateModel(x_train[[*colList]],x_test[[*colList]],y_train,y_test)

In [ ]:
for col in colList:
    evaluateModel(x_train[[col]],x_test[[col]],y_train,y_test)

In [ ]:
sinkModel = sm.OLS(y_train,sm.add_constant(x_train[[*colList]])).fit()

In [ ]:
sns.set(rc={'figure.figsize':(15, 10)})

sns.heatmap(data=df[['price',*colList]].corr(),annot=True)

In [ ]:
df[[*colList]]

In [ ]:
x_train

In [ ]:
cat_cols = ['condition','grade']
encoder = OneHotEncoder(handle_unknown='error',drop='first',categories='auto')
ct = ColumnTransformer(transformers=[('ohe', encoder, cat_cols)],remainder='passthrough')
ct.fit(x_train)
x_train_enc = ct.transform(x_train)
x_test_enc = ct.transform(x_train)

scaler = StandardScaler()
scaler.fit(x_train_enc)
x_train_scaled = scaler.transform(x_train_enc)
x_test_scaled = scaler.transform(x_test_enc)
x_train_sc_df = pd.DataFrame(x_train_scaled, columns= ct.get_feature_names())
x_train_sc_df

In [ ]:
train_df = pd.DataFrame(x_train_enc, columns= ct.get_feature_names())
train_df['target'] = y_train

In [ ]:
train_df.corr()['target'].map(abs).sort_values(ascending=False)

In [ ]:
len(y_train)

In [ ]:
lr_simple = LinearRegression()
lr_simple.fit(x_train_sc_df[['sqft_living']], y_train)

print(f"Train R2: {lr_simple.score(x_train_sc_df[['sqft_living']], y_train):.4f}")
x_test_sc_df = pd.DataFrame(x_test_scaled, columns= ct.get_feature_names())
print(f"Test R2: {lr_simple.score(x_test_sc_df[['sqft_living']], y_test):.4f}")

In [ ]:
cat_cols = ['condition', 'grade']

# create an encoder object
encoder = OneHotEncoder(handle_unknown='error',
                        drop='first',
                        categories='auto')

# Create an columntransformer object
ct = ColumnTransformer(transformers=[('ohe', encoder, cat_cols)],
                       remainder='passthrough')
ct.fit(x_train)
x_train_enc = ct.transform(x_train)
x_test_enc = ct.transform(x_test)

In [ ]:
scaler = StandardScaler()

scaler.fit(x_train_enc)

x_train_scaled = scaler.transform(x_train_enc)
x_test_scaled = scaler.transform(x_test_enc)

x_train_sc_df = pd.DataFrame(x_train_scaled, columns= ct.get_feature_names())
x_test_sc_df = pd.DataFrame(x_test_scaled, columns= ct.get_feature_names())

In [ ]:
evaluateModel(x_train_sc_df,x_test_sc_df,y_test,y_train)

In [ ]:
lr = LinearRegression()

# Run with our most strongly correlated feature
lr.fit(x_train_sc_df, y_train)

# Score on train
print(f"Train R2: {lr.score(x_train_sc_df, y_train):.4f}")

# Make a df version of test to score it too
x_test_sc_df = pd.DataFrame(x_test_scaled, columns= ct.get_feature_names())
print(f"Test R2: {lr.score(x_test_sc_df, y_test):.4f}")

In [ ]:
lr.predict()

ModuleNotFoundError: No module named 'utils.py'; 'utils' is not a package